This time we will correct the latest code version of the Female Daily web scrapping. In the first attempt I tried to scrape all the information with find_all in the first place. It implies that i can scrape information that is filled by "ALL" user. That's  because the program arange the record per element automatically as the order of the record itself. It means that if we have 1 user that doesn't fill 1 information it will disintegrate the order of the record.

This new version of code, it looks up the detail info per user first, so we get the informations per record correct. I also will return "None" or Missing value if the user didn't fill one or more element. 

# Import Modules

In [22]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
from datetime import datetime



In [23]:
#dir_path = os.getcwd()
startTime = datetime.now()

In [24]:
# Get the url (only 1 page)
hal = 2
url = 'https://reviews.femaledaily.com/products/lips/lipstick/dear-me-beauty/perfect-matte-lip-coat?cat=&cat_id=0&age_range=&skin_type=&skin_tone=&skin_undertone=&hair_texture=&hair_type=&order=newest&page={hal}'.format(hal=hal)
page = requests.get(url)

# TO USE SELENIUM

In [20]:
#impot module
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common.exceptions import TimeoutException
from bs4 import BeautifulSoup
import pandas as pd

In [21]:
chrome_options = Options()
chrome_options.add_experimental_option("prefs", {
  "download.default_directory": r"C:\Users\user\Downloads",
  "download.prompt_for_download": False,
  "download.directory_upgrade": True,
  "safebrowsing.enabled": False
})

In [19]:
driver = webdriver.Chrome("C:/Users/user/Downloads/chromedriver.exe",chrome_options=chrome_options)
driver.get(url)

WebDriverException: Message: Service C:/Users/user/Downloads/chromedriver.exe unexpectedly exited. Status code was: 0


In [25]:
#How to click Next Page 
driver.find_element_by_id("id_next_page").click

#PS I still have an issue with this line
#Is it just me or the next page button can not be clicked, HEEEELPPP

NameError: name 'driver' is not defined

In [26]:
# Wrangling HTML with BeautifulSoup

#Get Detail  info
soup = BeautifulSoup(page.content,'html.parser')
detailinfo=soup.find_all('div',class_='review-detail d-flex')


In [27]:
uname=[]
skintype=[]
oiltype=[]
shade=[]
tone=[]
age=[]
rating=[]
comment=[]
date=[]
rekomen=[]
usage_period=[]
purchase_point=[]

In [28]:
#Loop to get info per user review

for elem in detailinfo:
    #Get username
    Username = elem.find('p',class_='username')
    uname.append(Username.text)
    #Get Skintype 
    skin = elem.find('p',class_='skin').text
    
    #Parsing skin type
    skin2=skin.split(",")
    skintype.append(skin2)
    oiltype.append(skin2[0])
    shade.append(skin2[1])
    tone.append(skin2[2])
    age.append(skin2[3])
    
    #Get rating

    star = elem.find('span',class_='cardrv-starlist')
    bintang=star.find_all('i',{'class':lambda x: x and "icon-ic_big_star_full" in x.split()})
    rating.append(len(bintang))

    #Get Comment
    #PS I Can not click "Read More" on the comment section. Is it just me? :(
    komen = elem.find('p', class_="text-content")
    comment.append(komen.find('span').text)
    
    #Get Date review
    daterev=elem.find('p', class_="date review-date")
    date.append(daterev.text)
    
    #Get Recommended or not
    recinfo=elem.find('p', class_="recommend")
    if recinfo == None:
        rekomen.append("None")
    else :
        if "doesn't recommend" in recinfo.find('b').text:
            rekomen.append("Not Recommend")
        else:
            rekomen.append("Recommend")
            
    #Get Usage Period and Purchase Point
    infowrapper=elem.find('div', class_="information-wrapper")
    if infowrapper==None:
        usage_period.append("None")
        purchase_point.append("None")
    else :
        info2=infowrapper.find_all('b')
        usage_period.append(info2[0].text)
        purchase_point.append(info2[1].text)
            
 

In [29]:
df = pd.DataFrame(list(zip(uname, oiltype,shade, tone, age, rating,comment,date,rekomen,usage_period,purchase_point )), 
               columns =["username", "oiltype","shade", "tone", "age", "rating","comment","date","recommend","usage_period","purchase_point"])

In [30]:
print(df)

            username      oiltype          shade      tone       age  rating  \
0          archita97       Normal   Medium Light      Warm   19 - 24       5   
1            Nahla24  Combination         Medium      Warm   19 - 24       5   
2          isazarine  Combination         Medium   Neutral   19 - 24       4   
3     Nuzhiladwislma         Oily    Medium Dark   Neutral   19 - 24       5   
4  fatimahauliarahma  Combination         Medium   Neutral   19 - 24       4   
5          corona_xx         Oily   Medium Light   Neutral   19 - 24       5   
6       FahiraNisa41         Oily   Medium Light   Neutral   19 - 24       5   
7        yanuarizaim         Oily         Medium      Warm   19 - 24       5   
8           diviayur  Combination   Medium Light   Neutral   19 - 24       5   
9              agkla          Dry   Medium Light      Warm   19 - 24       3   

                                             comment         date  \
0  Produk DearMe pertama yang aku punya. Bagus ba.

In [31]:
print( datetime.now() - startTime)

0:00:24.125380
